# ThoroughBet Simulation


## Load necessary modules

In [1]:
import numpy as np

from thbmodel.utils import timestamp, YEAR, settings
from thbmodel.utils.arrayview import ArrayView, TimeseriesView

from thbmodel.prediction.models.fit_model import TSModel
from thbmodel.prediction.models.model_parameters import ModelParameters
from thbmodel.prediction.models.factor_management import FactorList
from thbmodel.prediction.models.preprocessing import load_slices

from thbmodel.prediction.models.prediction import factornames_trimmed
from thbmodel.prediction.models.parameters import factor_build_end

### Load data

In [2]:
settings.paths.join('racing_data_azd.av.bcolz')

'/home/oleg/thbmodel/racing_data/racing_data_azd.av.bcolz'

In [3]:
av = ArrayView.from_file(settings.paths.join('racing_data_azd.av.bcolz'))

In [4]:
tsav = load_slices()

### Preprocessing

In [5]:
pars = ModelParameters(av, oos_start=float(timestamp('2017-05-01')), depth=1, lmbd=10, verbose=False)

In [6]:
fl = FactorList.from_av(av, factornames_trimmed)

In [7]:
fl.preprocess(pars)
factors = fl.asmatrix()

In [8]:
from datetime import datetime
print 'factor_build_end  ',(datetime.utcfromtimestamp(pars.build_end))
print 'oos_start  ',(datetime.utcfromtimestamp(pars.oos_start))
print 'av_end  ',(datetime.utcfromtimestamp(pars._av_end))

factor_build_end   2016-03-01 00:00:00
oos_start   2017-05-01 00:00:00
av_end   2017-08-22 02:30:00


## Fit conditional logit model

In [9]:
tsmod = TSModel(factors, tsav, pars)

In [10]:
tsmod.fit_slices()
print 'LL................'
print tsmod.stats2.ll

thbmodel/prediction/models/fit_model.py:150: RuntimeWarning: divide by zero encountered in log
  stats, self.step2probs[sl] = self.concat_and_fit(strata, result, nonrunner, [fback, flay, np.log(probs)], ts_idx, valid2, verbose=False, step=2)


LL................
[[-1841.71211771 -2014.64783185 -2014.64783185]
 [-1846.09995945 -2010.17246104 -2010.17246104]
 [-1850.45158892 -2005.74888357 -2005.74888357]
 [-1852.84712372 -2001.06102244 -2001.06102244]
 [-1865.64087989 -1985.50064259 -1985.50064259]
 [-1876.45162575 -1977.58747572 -1977.58747572]
 [-1905.02665597 -1965.17813647 -1965.17813647]
 [-1915.51116973 -1957.48644079 -1957.48644079]
 [-1918.95666233 -1957.59955272 -1957.59955272]
 [-1922.13952769 -1952.82468284 -1952.82468284]
 [           nan            nan            nan]]


In [13]:
from thbmodel.prediction.tools.clustering import write_dic_to_simdata

#write_dic_to_simdata('new_factors_direction_new_code.p', new_model.step1probs, new_model.eff_coefs, oos, av =av)
write_dic_to_simdata('old_model_54components.p', tsmod.step1probs, tsmod.eff_coefs, pars.oos, av=av)

In [14]:
def new_factors_array (X, predict_mask =pars.model_mask):
    
    
    factors_new = np.zeros((X.shape[1], predict_mask.shape[0]))
    factors_new[:, predict_mask] = X.T
    
    return factors_new

In [15]:
# создаем дата фрейм из данных 
def DF(mask, factors, av, factors_names, other_names):
    
    import pandas as pd
    
    df = pd.DataFrame(data =factors[:, mask].T , columns = factors_names)
    for col in other_names :
        
        df[col] = av[col][mask]
        
    return df

In [17]:
import pandas as pd
pd.set_option('display.max_columns', 90)
n,m = factors.shape
col_names = ['f{}'.format(i) for i in range(1,n+1)]

df = DF (tsmod.params.model_mask, factors, av, col_names, ['event_id', 'runner_id', 'result', 'start_time', 
                                                           'jockey', 'trainer', 'prize'])
df['is1'] = tsmod.params.is1 [tsmod.params.model_mask]
df['is2'] = tsmod.params.is2 [tsmod.params.model_mask]
df['oos'] = tsmod.params.oos [tsmod.params.model_mask]


df.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,event_id,runner_id,result,start_time,jockey,trainer,prize,is1,is2,oos
0,0.275018,-2.136733,0.538481,-0.249552,0.000000,0.648431,1.956461,-0.350451,1.765546,0.041709,0.566215,0.186418,1.757950,-2.006400,0.350451,-0.146133,0.115240,0.000000,-0.181759,0.819832,0.128090,-2.007462,1.197907,0.000000,0.000000,-0.159251,1.247866,0.417520,0.819386,-2.570586,1.112466,-0.490569,0.000000,-0.298519,0.484388,0.314805,2.374919,0.0,-1.408942,0.120669,-0.940079,-0.407874,0.000000,0.772860,-0.118808,0.059176,1.022655,0.000000,0.102526,0.614107,0.895830,0.350452,-0.350448,0.310555,305646,355120,1,1.456841e+09,5598,5351,2911.0,True,True,False
1,0.275018,-1.165283,0.760248,-0.017615,-0.006324,-0.089657,-0.293469,0.028194,0.300362,0.301038,0.397894,0.302236,-0.428768,-0.862865,-0.027642,0.071812,0.081162,0.027632,-0.535525,-0.562630,-0.395142,-0.449051,-0.573235,0.128626,-0.078590,0.193096,0.903727,0.075159,0.722918,-0.696183,0.904835,-0.086411,0.446376,-0.200426,0.060539,0.679282,1.038988,0.0,-1.148013,-0.243849,-0.551588,0.006630,0.408662,0.772860,0.109651,4.479126,1.271640,0.300114,-0.190860,0.075975,0.124702,-0.027706,0.027719,0.062085,305646,419426,2,1.456841e+09,63,64,2911.0,True,True,False
2,0.275018,-1.019235,-0.620884,-0.054585,0.000418,0.343413,-0.586938,-0.087615,0.240985,0.514035,0.535161,0.260576,0.007447,-0.930042,0.087825,0.028731,-0.023594,0.119720,-0.567414,-0.434029,0.408481,-0.358223,-0.573235,0.219751,0.047266,0.359501,0.970164,-0.068555,0.169833,6.615283,-0.501394,0.872033,-0.311032,0.056299,1.078648,0.362859,1.002002,0.0,-0.598422,-0.243849,-0.337185,-0.026138,0.418632,0.962685,-0.037129,-1.127538,0.091651,0.216035,0.054865,0.740658,0.147065,0.087724,-0.089675,0.171093,305646,419427,3,1.456841e+09,8135,10791,2911.0,True,True,False
3,0.275018,-0.124600,0.648707,-0.129885,-0.036245,-0.044074,-0.586938,-0.025961,0.254463,0.217953,0.044346,0.076432,-0.222316,-0.642565,0.026061,0.118026,-0.091568,0.117425,0.299917,-1.446763,0.385371,0.554291,-0.573235,-0.034618,-0.059661,0.820580,-6.476758,-0.106326,0.214215,-0.610506,-0.115307,0.016959,-0.102501,0.087229,0.735120,0.293204,0.799318,0.0,-0.177459,-0.243849,-0.133236,0.097647,0.107262,0.772860,-0.113422,-0.390880,0.257840,0.115497,-0.160653,-0.021746,0.120533,0.026054,-0.026648,0.094833,305646,419428,5,1.456841e+09,29,408,2911.0,True,True,False
4,-0.687549,1.723630,-0.606623,-0.249552,0.000000,-0.202362,-0.313034,0.242896,0.144210,-0.155554,0.155252,-0.062139,-0.481582,2.027093,-0.242895,0.287826,-0.218564,0.271644,0.136029,1.945092,0.455057,1.366263,-0.573235,0.000000,0.206791,-0.022349,0.965077,0.032116,0.589176,-1.529798,-0.722877,-0.299929,0.000000,0.206902,-0.516705,-0.651000,-2.356321,0.0,0.473027,0.162551,-0.445529,-0.483757,0.000000,-2.027062,0.442830,-1.329056,-1.311230,0.000000,-0.035646,-0.209930,0.397297,-0.242895,0.242897,-0.079439,305646,401539,6,1.456841e+09,10807,10836,2911.0,True,True,False


In [18]:
from sklearn.decomposition import PCA
seed =7
n_components = 45
pca = PCA(n_components =n_components, random_state =seed)
name_pca = ['pca_f%s'%i for i in range(1, n_components+1)]

df_pca = pd.DataFrame(data = pca.fit_transform(df.loc[:,col_names]), columns =name_pca) 
    
pca_factors = new_factors_array(df_pca[name_pca])
pca_model = TSModel(pca_factors, tsav, pars)
pca_model.fit_slices()
    
print '{} PCA components'.format(n_components)
print 'LL................'
print pca_model.stats2.ll
print

45 PCA components
LL................
[[-1841.8231785  -2014.90686255 -2014.90686255]
 [-1846.20531733 -2010.34462411 -2010.34462411]
 [-1850.57395287 -2005.97976507 -2005.97976507]
 [-1852.98106992 -2001.31656051 -2001.31656051]
 [-1865.83606332 -1985.54833737 -1985.54833737]
 [-1876.70004601 -1977.35229545 -1977.35229545]
 [-1905.35743324 -1965.10990572 -1965.10990572]
 [-1916.09718726 -1957.5457566  -1957.5457566 ]
 [-1919.6128079  -1957.55827274 -1957.55827274]
 [-1922.96478944 -1952.43428967 -1952.43428967]
 [           nan            nan            nan]]



In [20]:
factors_names = np.array(['f_%s'%(i+1)  for i in range(pca_factors.shape[0])])

In [54]:
threshold =1.
names_factors =[]
_int_factors =[]
student_t_factors = []

for i in range(pca_factors.shape[0]):
    new_factors = pca_factors[i:,:] * pca_factors[i,:]
    new_factors = np.vstack((pca_factors, new_factors))
    print 'pca_f',(i+1)
    print new_factors.shape
    
    new_model = TSModel(new_factors, tsav, pars)
    new_model.fit_slices(fit_afresh=True)

    print 'll  for production factors %s'%(i+1)
    print new_model.stats1.ll
    print
    
    f = '*pca_f%s'%(i+1)
    factors_names = [x+f for x in name_pca[i:]]
    _int_factors += [(x,i) for x in range(i, pca_factors.shape[0])])
    names_factors.append(factors_names)
    mean_student_t = new_model.stats1.student_t[:,47:].mean(axis =0)
    student_t_factors.append(mean_student_t)
    mask_factors = (mean_student_t > threshold)
    
    
    names_important = np.array(factors_names)[mask_factors]
    print 'important factors len', len(names_important)
    print names_important
    
    print '......................'
    new_factors = np.vstack((pca_factors, new_factors[mask_factors,:]))
    new_model = TSModel(new_factors, tsav, pars)
    new_model.fit_slices(fit_afresh=True)
    print 
    print 'll   for only important factors '
    print new_model.stats1.ll

pca_f 1
(90, 1805375)
ll  for production factors 1
[[-1839.33224854 -1839.33224854 -2018.10169031]
 [-1843.60231941 -1843.60231941 -2013.85605178]
 [-1848.01578716 -1848.01578716 -2009.61348704]
 [-1850.38556713 -1850.38556713 -2004.854583  ]
 [-1863.23228397 -1863.23228397 -1989.26828745]
 [-1874.22210594 -1874.22210594 -1981.10559524]
 [-1903.03134973 -1903.03134973 -1968.81578522]
 [-1913.6505794  -1913.6505794  -1961.08624102]
 [-1917.29118644 -1917.29118644 -1961.11196898]
 [-1920.59622241 -1920.59622241 -1955.95153837]
 [           nan            nan            nan]]

important factors len 15
['pca_f2*pca_f1' 'pca_f7*pca_f1' 'pca_f12*pca_f1' 'pca_f15*pca_f1'
 'pca_f16*pca_f1' 'pca_f17*pca_f1' 'pca_f19*pca_f1' 'pca_f22*pca_f1'
 'pca_f23*pca_f1' 'pca_f25*pca_f1' 'pca_f27*pca_f1' 'pca_f29*pca_f1'
 'pca_f31*pca_f1' 'pca_f39*pca_f1' 'pca_f41*pca_f1']
......................


/home/oleg/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:31: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 90 but corresponding boolean dimension is 45



ll   for only important factors 
[[-1841.82890684 -1841.82890684 -2014.94005721]
 [-1846.21099955 -1846.21099955 -2010.43946739]
 [-1850.57975365 -1850.57975365 -2006.09111658]
 [-1852.98682071 -1852.98682071 -2001.45797987]
 [-1865.84285438 -1865.84285438 -1985.82734998]
 [-1876.70698083 -1876.70698083 -1977.75725377]
 [-1905.365866   -1905.365866   -1965.47771651]
 [-1916.10782275 -1916.10782275 -1957.81820743]
 [-1919.62350491 -1919.62350491 -1957.83553662]
 [-1922.97515669 -1922.97515669 -1952.63753475]
 [           nan            nan            nan]]
pca_f 2
(89, 1805375)
ll  for production factors 2
[[-1840.41404586 -1840.41404586 -2016.23741499]
 [-1844.82408331 -1844.82408331 -2011.65696619]
 [-1849.18895423 -1849.18895423 -2007.26830234]
 [-1851.66581885 -1851.66581885 -2002.59152312]
 [-1864.30761188 -1864.30761188 -1987.15651697]
 [-1875.06951224 -1875.06951224 -1979.40291524]
 [-1903.794197   -1903.794197   -1967.01489173]
 [-1914.32933272 -1914.32933272 -1959.46139794]
 [

/home/oleg/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:31: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 89 but corresponding boolean dimension is 44



ll   for only important factors 
[[-1841.85836931 -1841.85836931 -2014.93145956]
 [-1846.24232909 -1846.24232909 -2010.42412483]
 [-1850.61150878 -1850.61150878 -2006.07292637]
 [-1853.01942073 -1853.01942073 -2001.42864222]
 [-1865.87964471 -1865.87964471 -1985.75298758]
 [-1876.75102768 -1876.75102768 -1977.64902358]
 [-1905.40783079 -1905.40783079 -1965.35891241]
 [-1916.14388999 -1916.14388999 -1957.71619122]
 [-1919.66037651 -1919.66037651 -1957.71792068]
 [-1923.01459928 -1923.01459928 -1952.5241264 ]
 [           nan            nan            nan]]
pca_f 3
(88, 1805375)
ll  for production factors 3
[[-1840.42030238 -1840.42030238 -2015.98830191]
 [-1844.82434331 -1844.82434331 -2011.54052369]
 [-1849.13853694 -1849.13853694 -2007.17164486]
 [-1851.64097266 -1851.64097266 -2002.43622414]
 [-1864.40439501 -1864.40439501 -1986.56703323]
 [-1875.30754811 -1875.30754811 -1978.84781309]
 [-1903.84797396 -1903.84797396 -1966.3011331 ]
 [-1914.46179075 -1914.46179075 -1958.81192595]
 [

...


ll   for only important factors 
[[-1841.85878234 -1841.85878234 -2014.93524373]
 [-1846.24275609 -1846.24275609 -2010.42760097]
 [-1850.61195103 -1850.61195103 -2006.07540272]
 [-1853.01993134 -1853.01993134 -2001.4298553 ]
 [-1865.88018148 -1865.88018148 -1985.75202174]
 [-1876.75169137 -1876.75169137 -1977.64722236]
 [-1905.40863637 -1905.40863637 -1965.35683742]
 [-1916.14459515 -1916.14459515 -1957.71482538]
 [-1919.66111152 -1919.66111152 -1957.71667526]
 [-1923.01535745 -1923.01535745 -1952.52534289]
 [           nan            nan            nan]]
pca_f 44
(47, 1805375)
ll  for production factors 44
[[-1841.81513914 -1841.81513914 -2014.94229285]
 [-1846.19463827 -1846.19463827 -2010.46186871]
 [-1850.56746371 -1850.56746371 -2006.11337767]
 [-1852.98452531 -1852.98452531 -2001.44915692]
 [-1865.85572812 -1865.85572812 -1985.81620981]
 [-1876.72501914 -1876.72501914 -1977.68167722]
 [-1905.37023057 -1905.37023057 -1965.33140749]
 [-1916.08102807 -1916.08102807 -1957.62969839]


/home/oleg/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:31: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 47 but corresponding boolean dimension is 2



ll   for only important factors 
[[-1841.85875682 -1841.85875682 -2014.98388125]
 [-1846.24273737 -1846.24273737 -2010.47650883]
 [-1850.61193402 -1850.61193402 -2006.12449921]
 [-1853.01991858 -1853.01991858 -2001.47888237]
 [-1865.88018667 -1865.88018667 -1985.79929518]
 [-1876.75172136 -1876.75172136 -1977.69333472]
 [-1905.40872661 -1905.40872661 -1965.39671173]
 [-1916.14471628 -1916.14471628 -1957.74516679]
 [-1919.66123563 -1919.66123563 -1957.74486128]
 [-1923.01551494 -1923.01551494 -1952.54255394]
 [           nan            nan            nan]]
pca_f 45
(46, 1805375)
ll  for production factors 45
[[-1841.72555784 -1841.72555784 -2014.99930252]
 [-1846.10239831 -1846.10239831 -2010.49341442]
 [-1850.43412045 -1850.43412045 -2006.17141958]
 [-1852.84024327 -1852.84024327 -2001.54083241]
 [-1865.66221038 -1865.66221038 -1985.92284804]
 [-1876.49856411 -1876.49856411 -1977.87457002]
 [-1905.10432368 -1905.10432368 -1965.66253494]
 [-1915.84216366 -1915.84216366 -1958.02913441]


/home/oleg/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:31: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 46 but corresponding boolean dimension is 1



ll   for only important factors 
[[-1841.85878234 -1841.85878234 -2014.93524373]
 [-1846.24275609 -1846.24275609 -2010.42760097]
 [-1850.61195103 -1850.61195103 -2006.07540272]
 [-1853.01993134 -1853.01993134 -2001.4298553 ]
 [-1865.88018148 -1865.88018148 -1985.75202174]
 [-1876.75169137 -1876.75169137 -1977.64722236]
 [-1905.40863637 -1905.40863637 -1965.35683742]
 [-1916.14459515 -1916.14459515 -1957.71482538]
 [-1919.66111152 -1919.66111152 -1957.71667526]
 [-1923.01535745 -1923.01535745 -1952.52534289]
 [           nan            nan            nan]]


In [55]:
ss_TT = np.array([item for sublist in student_t_factors for item in sublist])
name_TT = np.array([item for sublist in names_factors for item in sublist])
ss_TT[ss_TT >2].shape

(70,)

In [56]:
def lists_to_list(_list):
    _x =[]
    for y in [x for x in _list]:
        _x = np.append(_x, y) 
    return _x

In [57]:
_names_factors = lists_to_list(names_factors)
student_factors = lists_to_list(student_t_factors)

In [58]:
_names_factors

array(['pca_f1*pca_f1', 'pca_f2*pca_f1', 'pca_f3*pca_f1', ...,
       'pca_f44*pca_f44', 'pca_f45*pca_f44', 'pca_f45*pca_f45'], 
      dtype='|S32')

In [59]:
st_name_factors = [(x, y) for x,y in zip(_names_factors, student_factors)]
st_name_factors.sort(key =lambda x: x[1], reverse =True)

In [60]:
st_name_factors

[('pca_f30*pca_f12', 4.0093495980333778),
 ('pca_f17*pca_f16', 3.3430922236443141),
 ('pca_f36*pca_f36', 3.1847181927487553),
 ('pca_f36*pca_f12', 3.0236772029847714),
 ('pca_f26*pca_f25', 2.8812223244373159),
 ('pca_f14*pca_f14', 2.881133854345943),
 ('pca_f19*pca_f16', 2.7913099213218349),
 ('pca_f36*pca_f26', 2.7910373967245841),
 ('pca_f29*pca_f12', 2.7897011537947316),
 ('pca_f25*pca_f19', 2.7885353233737793),
 ('pca_f14*pca_f4', 2.7668803088331644),
 ('pca_f45*pca_f16', 2.7155497725354754),
 ('pca_f2*pca_f1', 2.6922745282347225),
 ('pca_f8*pca_f2', 2.6561880285077604),
 ('pca_f38*pca_f25', 2.5857033721238722),
 ('pca_f45*pca_f45', 2.5855482715160849),
 ('pca_f36*pca_f15', 2.5662333350968853),
 ('pca_f44*pca_f17', 2.5493759307867361),
 ('pca_f34*pca_f14', 2.5472255833196402),
 ('pca_f25*pca_f22', 2.5092425198890767),
 ('pca_f22*pca_f9', 2.4994972421661772),
 ('pca_f42*pca_f11', 2.4790187071063703),
 ('pca_f41*pca_f7', 2.4763490452845134),
 ('pca_f22*pca_f22', 2.4738334211334942),


In [30]:
def int_factors(_list):
    list_factors =[]
    _list = [x.split('*') for x in _list]
    for _x_str in _list:
        list_factors.append([int(filter(str.isdigit,x)) for x in _x_str])
    return np.array(list_factors)

In [61]:
def list_factor(st_name_factors, top = 20):
    list_factors =[]
    _list = [x[0] for x in st_name_factors[:top]]
    _list = [x.split('*') for x in _list]
    for _x_str in _list:
        list_factors.append([int(filter(str.isdigit,x)) for x in _x_str])
    return np.array(list_factors)

In [69]:
factors_numbers = list_factor(st_name_factors, 10) -1

In [77]:
def polinom_factors(factors_numbers, top =10, factors =factors):
    
    new_factors = np.full((top, factors.shape[1]), np.nan)
    for i,j in enumerate(factors_numbers):
        new_factors[i,:] = factors[j[0],:]*factors[j[1],:]
    return new_factors

In [82]:
def important_polinoms_factors(names_factors, student_t_factors, top =30, factors =factors):
    
    #list of lists in one list
    names_factors = lists_to_list(names_factors)
    student_t_factors = lists_to_list(student_t_factors)
    #sort by values of student_t_factors, with remember of names factors
    names_student_factors = [(x, y) for x,y in zip(names_factors, student_factors)]
    names_student_factors.sort(key =lambda x: x[1], reverse =True)
    #list of top factors in int type
    factors_numbers = list_factor(names_student_factors, top =top) -1
    
    return polinom_factors(factors_numbers, top =top, factors =factors)

In [91]:
new_factors = np.vstack((pca_factors, 
            important_polinoms_factors(names_factors, student_t_factors, 5, pca_factors)))
new_factors.shape

(50, 1805375)

In [97]:
new_model = TSModel(new_factors, tsav, pars)

new_model.fit_slices(fit_afresh=True)

print 'll'
print new_model.stats1.ll

ll
[[-1840.16791435 -1840.16791435 -2014.37865554]
 [-1844.55461153 -1844.55461153 -2009.97149907]
 [-1848.84877571 -1848.84877571 -2005.72041167]
 [-1851.26792442 -1851.26792442 -2001.17716753]
 [-1864.05845882 -1864.05845882 -1985.64674071]
 [-1874.88170041 -1874.88170041 -1977.70550239]
 [-1903.41544102 -1903.41544102 -1965.7915072 ]
 [-1913.90161721 -1913.90161721 -1958.45556158]
 [-1917.34963979 -1917.34963979 -1958.457847  ]
 [-1920.8535894  -1920.8535894  -1953.25638537]
 [           nan            nan            nan]]


In [93]:
from thbmodel.prediction.tools.clustering import write_dic_to_simdata
write_dic_to_simdata('model_new_factors.p', new_model.step1probs, new_model.eff_coefs, new_model.params.oos, av=av)

In [85]:
old_model = TSModel(factors, tsav, pars)
old_model.fit_slices(fit_afresh=True)

print 'll'
print old_model.stats1.ll

ll
[[-1841.71134254 -1841.71134254 -2014.22627609]
 [-1846.09933267 -1846.09933267 -2009.80810427]
 [-1850.45108524 -1850.45108524 -2005.42403158]
 [-1852.84651257 -1852.84651257 -2000.77815071]
 [-1865.64182041 -1865.64182041 -1985.37827819]
 [-1876.45454829 -1876.45454829 -1977.56090486]
 [-1905.03126923 -1905.03126923 -1965.26265245]
 [-1915.51635675 -1915.51635675 -1957.59727056]
 [-1918.96163692 -1918.96163692 -1957.71061385]
 [-1922.14635182 -1922.14635182 -1952.90889816]
 [           nan            nan            nan]]


In [45]:
from thbmodel.prediction.tools.clustering import write_dic_to_simdata
write_dic_to_simdata('old_model.p', old_model.step1probs, old_model.eff_coefs, old_model.params.oos, av=av)

In [ ]:
len(np.unique(av.event_id[pars.is1])), len(np.unique(av.event_id[pars.oos]))

In [ ]:
import pickle
with open('old_model.pickle', 'wb') as f:
    pickle.dump(tsmod, f)

In [ ]:
import torch

In [ ]:
x = torch.FloatTensor(1024, 1024).uniform_()

In [ ]:
x = x.cuda()

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
%%timeit
a = torch.FloatTensor(10000, 10000).uniform_()
b = torch.FloatTensor(10000, 10000).uniform_()
a = a.cuda()
b = b.cuda()
c = a.mul_(b)

In [ ]:
%%timeit
a = torch.FloatTensor(10000, 10000).uniform_()
b = torch.FloatTensor(10000, 10000).uniform_()
c = a.mul_(b)